## Pandas Optimization - Best Practice

- Downcast datatypes
- Faster Lookups
- Avoid creating copy
- Filter before merge/join/concat

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import time
import pandas as pd
import numpy as np

### Downcasting

Downcasting involves converting columns to more memory-efficient data types. For instance, you can convert float64 to float32 or int64 to int32 when the values fit within the smaller data types, saving memory without losing information.

In [4]:
# Create a DataFrame with large data types
df = pd.DataFrame({
    'A': np.random.randint(0, 100, size=1000000),  # int64
    'B': np.random.rand(1000000)                    # float64
})

# Check memory usage before downcasting
df.info()
print()
# Downcast the integer and float columns
df['A'] = pd.to_numeric(df['A'], downcast='integer')  # Downcast to int8
df['B'] = pd.to_numeric(df['B'], downcast='float')    # Downcast to float32

# Check memory usage after downcasting
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   A       1000000 non-null  int64  
 1   B       1000000 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 15.3 MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   A       1000000 non-null  int8   
 1   B       1000000 non-null  float32
dtypes: float32(1), int8(1)
memory usage: 4.8 MB


In [5]:
df = pd.DataFrame({
    'city': ['New York', 'Los Angeles', 'New York', 'Chicago'] * 1000
})

df.info()
print()
# Before optimization
print(df['city'].memory_usage())

# Convert to categorical
df['city'] = df['city'].astype('category')
print(df['city'].memory_usage())
print()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   city    4000 non-null   object
dtypes: object(1)
memory usage: 31.4+ KB

32128
4260

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   city    4000 non-null   category
dtypes: category(1)
memory usage: 4.2 KB


### Faster Lookups

.iloc and .loc can be used for efficient and faster lookups

In [6]:
# Create a large DataFrame
df = pd.DataFrame({
    'key': range(1, 100000001),
    'value': range(100000000)
})

start_time = time.time()
# Fast lookup using the index
result = df.iloc[500000]  # This is faster than using .iloc or searching without an index
end_time = time.time()

print(f"Time taken: {end_time-start_time:.4f}")

start_time = time.time()
# Fast lookup using the index
result = df.loc[500000]  # This is faster than using .iloc or searching without an index
end_time = time.time()

print(f"Time taken: {end_time-start_time:.4f}")

# print(result)

Time taken: 0.0014
Time taken: 0.0002


### Filter

When merging or joining large DataFrames, it’s beneficial to filter them down to only the necessary rows beforehand. This reduces the size of the DataFrames involved in the operation, improving performance.

In [13]:
df1 = pd.DataFrame({
    'key': ['A', 'B', 'C', 'D'],
    'value1': [1, 2, 3, 4]
})

df2 = pd.DataFrame({
    'key': ['B', 'C', 'D', 'E'],
    'value2': [5, 6, 7, 8]
})

# Filter DataFrames before merging
df1_filtered = df1[df1['value1'] > 1]  # Keep only rows where value1 > 1
df2_filtered = df2[df2['key'].isin(df1_filtered['key'])]  # Keep only matching keys

# Perform the merge
merged_df = pd.merge(df1_filtered, df2_filtered, on='key')
print(merged_df)

  key  value1  value2
0   B       2       5
1   C       3       6
2   D       4       7


### Avoid creating copy

inplace parameter ensures the changes are incorporated in the same dataframe instead of creating a new one.

In [30]:
df = pd.DataFrame({
    'A': np.random.randint(0, 100, size=100),
    'B': np.random.randint(0, 100, size=100)
})

df.drop(columns=['B'], inplace=True)
df.head()

,A
0,66
1,29
2,54
3,93
4,85
